In [1]:
# prompt: 用url 爬蟲

import requests
from bs4 import BeautifulSoup
import re
import random
import time
from urllib.parse import quote
from tqdm import tqdm, trange
import csv
from datetime import date
import os,os.path
import json

In [2]:
def read_url(url):
  #根據url建立連線
  try:
    response = requests.get(url)
    response.raise_for_status()  # 檢查 HTTP 狀態碼是否為 200
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup
  except requests.exceptions.RequestException as e:
    print(f"爬取 URL 失敗: {e}")
    return None




In [3]:
#安裝selenium、chromium-chromedriver驅動及更新
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,032 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
G

In [4]:
def selenium_get_Code_104(url):
    #chrome_options = Options() # 啟動無頭模式
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    # Change 'options' to 'chrome_options' and remove 'executable_path'
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    #要睡一下，有時候網路太爛他會抓到還沒跑完的網頁資料
    time.sleep(1)
    save = driver.page_source

    driver.quit()#關閉瀏覽器
    soup = BeautifulSoup(save, "html.parser")
    page_select = soup.select_one('div.high-light.multiselect__content-wrapper > ul > li:nth-child(1) > span')
    text = page_select.text
    txtre =re.search(r'/\s*(\d+)',text)
    #用re取"第1/142頁"中的142
    page = txtre.group(1)
    return page

In [ ]:
import time
from datetime import date
import csv
from urllib.parse import quote
from tqdm import tqdm
import os
import re
import logging
import random
from google.colab import drive
import requests
import bs4
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util import Retry
drive.mount('/content/drive')

# 設置日誌
logging.basicConfig(filename='scraping.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

today = date.today()

# 關鍵字列表
# key_texts = ["軟體工程師", "Software Developer", "通訊軟體工程師", "韌體工程師", "Firmware Engineer", "軟體測試人員", "QA Engineer", "BIOS工程師", "BIOS Engineer", "CIM工程師", "MES工程師", "網站程式設計師", "Web Developer"]  # 可以根據需要添加更多關鍵字
key_texts = ["DevOps工程師", "區塊鏈工程師", "Blockchain Engineer"]  # 可以根據需要添加更多關鍵字

def selenium_get_Code_104(url):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    time.sleep(1)
    save = driver.page_source

    driver.quit()
    soup = BeautifulSoup(save, "html.parser")
    page_select = soup.select_one('div.high-light.multiselect__content-wrapper > ul > li:nth-child(1) > span')
    text = page_select.text
    txtre = re.search(r'/\s*(\d+)', text)
    page = txtre.group(1)
    return page

def scrape_jobs(key_txt):
    # URL編碼關鍵字
    key = quote(key_txt)
    # 設置CSV文件路徑
    path_csv = f"/content/drive/My Drive/colab data/{today}_{key_txt}_104人力銀行.csv"
    if not os.path.isdir('jobs_csv'):
        os.mkdir('jobs_csv')
        logging.info('建立jobs_csv資料夾完成')

    # 創建CSV文件並寫入標題
    with open(path_csv, mode='w', newline='', encoding='utf-8') as employee_file:
        employee_writer = csv.writer(
            employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        employee_writer.writerow(['日期', '工作名稱', '公司名稱', '職務類別', '薪資', '工作內容', '地區',
                                 '經歷', '學歷', '兩周內應徵人數', '文章編號', '工作網址', '產業別', '擅長工具', '工作技能', '科系要求', '其他條件'])

    max_retries = 5
    try:
        url = f"https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword={key}&order=15&asc=0&page=1&mode=s&jobsource=2018indexpoc"

        # 使用 selenium_get_Code_104 函數獲取總頁數
        total_pages = int(selenium_get_Code_104(url))
        get_sum_page = min(total_pages, 150)  # 限制最多爬取150頁

        logging.info(f'開始爬取關鍵字: {key_txt}, 共有：{get_sum_page} 頁')

        for page in tqdm(range(1, get_sum_page + 1), desc=f"爬取 {key_txt}"):
            url = f"https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword={key}&order=15&asc=0&page={page}&mode=s&jobsource=2018indexpoc"

            for attempt in range(max_retries):
                try:
                    soup = read_url(url)
                    if soup is None:
                        raise Exception("Failed to retrieve page content")
                    break
                except Exception as e:
                    if attempt == max_retries - 1:
                        logging.error(f"爬取頁面失敗: {url}, 錯誤: {e}")
                        continue
                    sleep_time = exponential_backoff(attempt)
                    time.sleep(sleep_time)

            if soup is None:
                continue

            for title_1 in soup.select('.job-summary'):
                try:
                    # 提取各種職位信息
                    date = '廣告' if title_1.select(
                        '.col-auto.date')[0].select('i') else title_1.select('.col-auto.date')[0].get_text()
                    area = title_1.select(
                        '.info-tags.gray-deep-dark')[0].find('a').get_text()
                    experience = title_1.select(
                        '.info-tags.gray-deep-dark')[0].find_all('a')[1].get_text()
                    education = title_1.select('.info-tags.gray-deep-dark')[0].find_all('a')[2].get_text(
                    ) if len(title_1.select('.info-tags.gray-deep-dark')[0].find_all('a')) > 2 else ""
                    title_url = title_1.select(
                        '.info-job__text.jb-link.jb-link-blue.jb-link-blue--visited.h2')[0]['href']
                    title_str = title_url.split('?')[0].split('/')[-1]
                    title = title_1.select(
                        '.info-job__text.jb-link.jb-link-blue.jb-link-blue--visited.h2')[0]['title']
                    company_name = title_1.select(
                        '.info-company__text.jb-link.jb-link-blue.jb-link-blue--visited.h4')[0].get_text()

                    salary_element = title_1.select(
                        '.info-tags.gray-deep-dark')[0].find_all('a')
                    salary = salary_element[3].get_text() if len(
                        salary_element) > 3 else "待遇面議"
                    if salary != "待遇面議":
                        salary = re.search(
                            r'\d+.\d+', salary).group() if re.search(r'\d+.\d+', salary) else "0"

                    people = title_1.select('.action-apply__range.d-flex.text-center.align-items-center')[
                        0]['title'] if title_1.select('.action-apply__range.d-flex.text-center.align-items-center') else ""
                    industry = title_1.select(
                        '.info-company-addon-type.text-gray-darker.font-weight-bold')[0].get_text()
                    industry = industry if "業" in industry else "無"

                    # 爬取工作網址內的內容
                    detail_soup = read_url(title_url)

                    introduction, tools, skills, job_categories, major_requirement, other_requirements = extract_job_details(
                        detail_soup)

                    # 準備寫入CSV的行數據
                    row = [date, title, company_name, job_categories, salary, introduction, area, experience, education,
                           people, title_str, title_url, industry, tools, skills, major_requirement, other_requirements]

                    # 將數據寫入CSV文件
                    with open(path_csv, mode='a', newline='', encoding='utf-8') as employee_file:
                        employee_writer = csv.writer(
                            employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                        employee_writer.writerow(row)

                    time.sleep(random.uniform(1, 3))  # 隨機延遲1-3秒，避免過快請求
                except Exception as e:
                    logging.error(f"處理職位資訊時發生錯誤: {e}")
                    continue

        logging.info(f"完成爬取關鍵字: {key_txt}")
    except Exception as e:
        logging.error(f"爬蟲過程中發生錯誤: {e}")

def extract_job_details(detail_soup):
    introduction = ""
    tools = []
    skills = []
    job_categories = []
    major_requirement = ""
    other_requirements = ""

    if detail_soup:
        job_description = detail_soup.select_one('p.job-description__content')
        if job_description:
            introduction = job_description.get_text(strip=True)
        else:
            introduction = "未提供工作內容"

        list_rows = detail_soup.select('div.list-row.row.mb-2')
        tools = []
        skills = []
        job_categories = []
        major_requirement = "未指定"
        other_requirements = "不拘"

        for row in list_rows:
            header = row.select_one('div.col.p-0.mr-4.list-row__head h3')
            if header:
                header_text = header.get_text().strip()
                data_block = row.select_one('div.col.p-0.list-row__data')

                if data_block:
                    if "擅長工具" in header_text:
                        tools = [u.get_text().strip()
                                 for u in data_block.select('a.tools u')]
                    elif "工作技能" in header_text:
                        skills = [u.get_text().strip()
                                  for u in data_block.select('a.skills u')]
                    elif "職務類別" in header_text:
                        job_categories = [u.get_text().strip()
                                          for u in data_block.select('u')]
                    elif "科系要求" in header_text:
                        major_element = data_block.select_one('div.t3.mb-0')
                        major_requirement = major_element.get_text(
                            strip=True) if major_element else "未指定"
                    elif "其他條件" in header_text:
                        other_element = data_block.select_one(
                            'div.col.p-0.job-requirement-table__data p.t3.m-0') or data_block.select_one('div.t3.mb-0')
                        other_requirements = other_element.get_text(
                            strip=True) if other_element else "不拘"

        # 將列表轉換為字符串，並處理空值
        tools = ', '.join(tools) if tools else "不拘"
        skills = ', '.join(skills) if skills else "不拘"
        job_categories = ', '.join(job_categories) if job_categories else "未指定"
        other_requirements = "不拘" if other_requirements.strip() in [
            "", "未填寫"] else other_requirements

    return introduction, tools, skills, job_categories, major_requirement, other_requirements


# 主程序
for key_txt in key_texts:
    scrape_jobs(key_txt)

logging.info("所有關鍵字爬蟲完成")


def exponential_backoff(attempt):
    return min(30, (2 ** attempt) + random.uniform(0, 1))


def read_url(url):
    session = create_retry_session()
    headers = get_random_headers()
    try:
        response = session.get(url, headers=headers)
        response.raise_for_status()
        return bs4.BeautifulSoup(response.text, 'html.parser')
    except requests.exceptions.RequestException as e:
        if e.response is not None and e.response.status_code == 429:
            logging.warning("遇到 429 錯誤，嘗試使用新的 header")
            headers = get_random_headers()  # 獲取新的隨機 header
            try:
                response = session.get(url, headers=headers)
                response.raise_for_status()
                return bs4.BeautifulSoup(response.text, 'html.parser')
            except requests.exceptions.RequestException as e:
                logging.error(f"使用新 header 後仍然爬取 URL 失敗: {e}")
                return None
        else:
            logging.error(f"爬取 URL 失敗: {e}")
            return None

def create_retry_session(retries=3, backoff_factor=0.3):
    retry_strategy = Retry(
        total=retries,
        backoff_factor=backoff_factor,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["HEAD", "GET", "OPTIONS", "POST", "PUT", "DELETE"]
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session = requests.Session()
    session.mount("https://", adapter)
    session.mount("http://", adapter)
    return session

def get_random_headers():
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36',
        'Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1'
    ]
    return {
        'User-Agent': random.choice(user_agents),
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Cache-Control': 'max-age=0'
    }


Mounted at /content/drive


爬取 Blockchain Engineer:   7%|▋         | 11/150 [12:52<2:34:21, 66.63s/it]

In [ ]:
url='https://www.104.com.tw/jobs/search/?jobsource=index_s&keyword=data&mode=s&page=3&order=15'
soup=read_url(url)
for title_1 in soup.select('.job-summary'):
  a= title_1.select('.info-company-addon-type.text-gray-darker.font-weight-bold')[0].get_text()



  print(a)